In [1]:
import torch
import wandb
import diffusion_pde as dpde
from pathlib import Path
from hydra import initialize, compose
from omegaconf import OmegaConf
from wandb.apis.public.runs import Runs
from wandb.apis.public.artifacts import RunArtifacts

In [2]:
API = wandb.Api()
wandb_cfg = OmegaConf.load("../conf/train.yaml").wandb

filters = {}#{"job_type": "train"} #{"tags": "fine-tune"}

runs = Runs(
    client=API.client,
    entity=wandb_cfg.entity,
    project=wandb_cfg.project,
    filters=filters
)

for i, run in enumerate(runs):
    print(f"Index: {i} - Run ID: {run.id}, Name: {run.name}")

Index: 0 - Run ID: jjj9pelv, Name: heat-logt/joint/unet-v2
Index: 1 - Run ID: 0pp5ahlc, Name: heat-logt/joint/unet-v2/fine-tune
Index: 2 - Run ID: 7revrf5r, Name: heat-no-cond/joint/unet-small
Index: 3 - Run ID: 1ml9aurp, Name: heat-lineart/joint/unet-v2
Index: 4 - Run ID: qbd7omen, Name: heat-lineart/joint/unet-v2/fine-tune
Index: 5 - Run ID: s42g9h6h, Name: heat_no_cond_test/dataset
Index: 6 - Run ID: xo3rg6u8, Name: heat-no-cond/joint/unet-small/test
Index: 7 - Run ID: s6svnr60, Name: heat-no-cond/joint/unet-small/test
Index: 8 - Run ID: 6usadlpw, Name: heat-logt/joint/unet-v2/test
Index: 9 - Run ID: 8dp00llj, Name: heat-no-time/joint/unet-small
Index: 10 - Run ID: 9vsc0osd, Name: heat-no-time/joint/unet-small/test
Index: 11 - Run ID: hsjrwptv, Name: heat-no-time/joint/unet-small/test
Index: 12 - Run ID: beuvkdus, Name: heat-no-time/joint/unet-v2
Index: 13 - Run ID: ehi00g22, Name: heat-no-time/joint/unet-v2/test
Index: 14 - Run ID: 5gf2x0m8, Name: llg-no-time/joint/unet-v2
Index: 1

In [3]:
arts = RunArtifacts(client=API.client, run=runs[15])
for art in arts:
    if art.type == "dataset":
        _ = art.download(root=f"../data/")

wandb: Downloading large artifact 'all-test-datasets:v0', 1277.43MB. 7 files...
wandb:   7 of 7 files downloaded.  
Done. 00:00:17.9 (71.2MB/s)


In [5]:
data_path = Path("/home/s204790/dynamical-pde-diffusion/data/llg_no_time.hdf5")
data_path_test = Path("/home/s204790/dynamical-pde-diffusion/data/llg_no_time_test.hdf5")
print(data_path.exists(), data_path_test.exists())
data_path.stem

True True


'llg_no_time'

In [ ]:
with initialize(config_path="../conf", version_base=None):
    cfg = compose(config_name="train", overrides=[])

wandb_kwargs = OmegaConf.to_container(cfg.wandb, resolve=True)
wandb_kwargs["name"] = f"test-datasets/dataset"
wandb_kwargs

{'entity': 'philiphohwy-danmarks-tekniske-universitet-dtu',
 'project': 'dynamical-pde-diffusion-final',
 'dir': '/home/s204790/logs',
 'name': 'heat-test-datasets/dataset',
 'mode': 'online'}

In [9]:
data_dir = dpde.utils.get_repo_root() / "data"

files = [p for p in data_dir.iterdir() if p.is_file() and "test" in p.stem]

[f.stem for f in files]

['heat_lineart_test',
 'llg_no_time_test',
 'heat_logt_test',
 'llg_test',
 'heat_no_time_test',
 'heat_no_cond_test',
 'heat_no_cond_test2']

In [10]:
with wandb.init(**wandb_kwargs) as run:
    artifact = wandb.Artifact(name="all-test-datasets", type="dataset")
    for file_path in files:
        artifact.add_file(str(file_path))
    artifact.metadata = {
        "description": "all test datasets for LLG and heat equation.",
    }
    run.log_artifact(artifact)

In [ ]:
api = wandb.Api()